In [12]:
import utils
import importlib

importlib.reload(utils)
from utils import *
pd.set_option('display.max_colwidth', None)

In [13]:
excel_path = "/home/viktoria/Downloads/ARBOLES ALIMENTOS EJERCICIO KABELLI.xlsx"
# all data
df_all = load_all_sheets(excel_path)
df_all.shape

(2381, 4)

In [14]:
excel_path = "/home/viktoria/Downloads/ARBOLES ALIMENTOS EJERCICIO KABELLI.xlsx"

confident, needs_review = process_excel_for_duplicates(
    excel_path,
    confidence_threshold=93,
    low_confidence_threshold=88
)


In [15]:
confident.shape

(59, 10)

In [16]:
needs_review.shape

(41, 10)

In [17]:
confident.head(30)

,Marca,Nombre SKU 1,SKU 1,Sheet 1,Nombre SKU 2,SKU 2,Sheet 2,Similarity,Numbers 1,Numbers 2
0,COLISEO,AZUCAR 1 KG COLISEO,22960,Surtiventas Arbol_24_25,AZUCAR 1 KG. COLISEO,71978,Prisur Arbol_24_25,100,[1.0],[1.0]
1,COLISEO,ATUN DESMENUZADO AGUA 170 GR COLISEO,97840,PRISA Arbol_24_25,ATUN DESMENUZADO AGUA 170 GR COLISEO,85102,Prisur Arbol_24_25,100,[170.0],[170.0]
2,COLISEO,ATUN DESMENUZADO AGUA 170 GR COLISEO,97840,PRISA Arbol_24_25,ATUN DESMENUZADO AGUA 170 GR COLISEO,75969,Surtiventas Arbol_24_25,100,[170.0],[170.0]
3,BELMONT,ACEITE COCINA 250 CC. VEGETAL BELMONT,79094,Surtiventas Arbol_24_25,ACEITE COCINA 250 CC. VEGETAL BELMONT,19256,Prisur Arbol_24_25,100,[250.0],[250.0]
4,COLISEO,POROTO 1 KG. HALLADO COLISEO,75760,Surtiventas Arbol_24_25,POROTO 1 KG. COLISEO HALLADO,85096,Prisur Arbol_24_25,100,[1.0],[1.0]
5,COLISEO,LENTEJA 1 KG. 6 MM COLISEO,75758,Surtiventas Arbol_24_25,LENTEJA 1 KG 6 MM COLISEO,85092,Prisur Arbol_24_25,100,"[1.0, 6.0]","[1.0, 6.0]"
6,COLISEO,ATUN DESMENUZADO AGUA 170 GR COLISEO,75969,Surtiventas Arbol_24_25,ATUN DESMENUZADO AGUA 170 GR COLISEO,85102,Prisur Arbol_24_25,100,[170.0],[170.0]
7,COLISEO,AZUCAR 1 KG. COLISEO,22960,Prinorte Arbol_24_25,AZUCAR 1 KG. COLISEO,71978,Prisur Arbol_24_25,100,[1.0],[1.0]
8,BELMONT,ACEITE COCINA 250 CC. VEGETAL BELMONT,79094,Prinorte Arbol_24_25,ACEITE COCINA 250 CC. VEGETAL BELMONT,19256,Prisur Arbol_24_25,100,[250.0],[250.0]
9,IANSA,AZUCAR 5 GR X 800 SACHET APROX. IANSA.,99845,PRISA Arbol_24_25,AZUCAR 5 GR X 800 SACHET APROX. IANSA,14395,Prisur Arbol_24_25,100,"[5.0, 800.0]","[5.0, 800.0]"


In [18]:
confident.tail(30)

,Marca,Nombre SKU 1,SKU 1,Sheet 1,Nombre SKU 2,SKU 2,Sheet 2,Similarity,Numbers 1,Numbers 2
29,KRYZPO,PAPA FRITA 130 GR KRYZPO CREMA CEBOLLA,92303,Surtiventas Arbol_24_25,PAPA FRITA 130 GR KRYZPO CREMA CEBOLLA,45521,Prisur Arbol_24_25,100,[130.0],[130.0]
30,HOMAR,PALMITO 400 GR RODAJA TARRO HOMAR,72720,Surtiventas Arbol_24_25,PALMITOS 400 GR RODAJA TARRO HOMAR,82606,Prisur Arbol_24_25,99,[400.0],[400.0]
31,ALOE WIN,BEBIDA ALOE VERA 500 ML PI¥A ALOE WIN,35160,Surtiventas Arbol_24_25,BEBIDA ALOE VERA 500 ML PINA ALOE WIN,72475,Prisur Arbol_24_25,99,[500.0],[500.0]
32,SAN JOSE,ATUN LOMITOS ACEITE 160 GR SAN JOSE,72495,Surtiventas Arbol_24_25,ATUN LOMITO ACEITE 160 GR SAN JOSE,74855,Prisur Arbol_24_25,99,[160.0],[160.0]
33,NESTLE,CREMA DE LECHE NESTLE 157 GRS.,85155,PRISA Arbol_24_25,CREMA DE LECHE 157 GR NESTLE,99862,Prisur Arbol_24_25,98,[157.0],[157.0]
34,HOMAR,ATUN LOMITOS ACEITE 140 GR HOMAR,72423,Surtiventas Arbol_24_25,ATUN LOMITO ACEITE 140 GR HOMAR,82598,Prisur Arbol_24_25,98,[140.0],[140.0]
35,HOMAR,ATUN LOMITOS AGUA 140 GR HOMAR,72424,Surtiventas Arbol_24_25,ATUN LOMITO AGUA 140 GR HOMAR,82599,Prisur Arbol_24_25,98,[140.0],[140.0]
36,COSTA,CEREAL BARRA COSTA FRUTAS + YOGURT X 20 UDS,36035,PRISA Arbol_24_25,CEREAL BARRA COSTA FRUTOS + YOGURT X 20 UDS,70941,Prisur Arbol_24_25,98,[20.0],[20.0]
37,SAN JOSE,ATUN LOMITOS AGUA 160 GR SAN JOSE,78799,Surtiventas Arbol_24_25,ATUN AGUA LOMITO 160 GR SAN JOSE,74854,Prisur Arbol_24_25,98,[160.0],[160.0]
38,COSTA,GALLETA COSTA DELI COOKIES CLASICA 240 GRS,95772,PRISA Arbol_24_25,GALLETA COSTA 240 GR DELI COOKIE CLASICA,79322,Surtiventas Arbol_24_25,98,[240.0],[240.0]


In [19]:
needs_review.head(41)
# Aparecen poductos que ya pueden ser diferentes como
#CEREAL BARRA COSTA FRUTOS ROJOS + YOGURT X 20 UDS	 -  CEREAL BARRA COSTA FRUTOS ROJOS (No tiene Yogurt)
# SALSA DE SOYA SUAVE 165 CC GOURMET -  SALSA DE SOYA SUAVE 165 CC GOURMET (tiene Sauve)

,Marca,Nombre SKU 1,SKU 1,Sheet 1,Nombre SKU 2,SKU 2,Sheet 2,Similarity,Numbers 1,Numbers 2
0,LUCCHETTI,FIDEO 400 GR TALLARIN 77 LUCCHETTI,24295,Surtiventas Arbol_24_25,PASTA FIDEO 400 GR LUCCHETTI TALLARIN 77,75555,Prisur Arbol_24_25,92,"[400.0, 77.0]","[400.0, 77.0]"
1,WASIL,ARVEJA 340 CC. WASIL TETRA,57039,Surtiventas Arbol_24_25,ARVEJA 340 GR. WASIL TETRA,57031,Prisur Arbol_24_25,92,[340.0],[340.0]
2,COLISEO,FIDEO 400 GR SPAGUETTI COLISEO SPECIALE,54052,Surtiventas Arbol_24_25,PASTA FIDEO 400 GR COLISEO SPAGUETTI SPECIAL,72501,Prisur Arbol_24_25,92,[400.0],[400.0]
3,NESCAFE,CAFE INST. NESCAFE TRADICION 400 GR POLVO TARRO,12596,PRISA Arbol_24_25,CAFE 400 GR NESCAFE TRADICIONAL POLVO TARRO,89295,Prisur Arbol_24_25,92,[400.0],[400.0]
4,COSTA,CEREAL BARRA COSTA FRUTOS ROJOS + YOGURT X 20 UDS,36035,PRISA Arbol_24_25,CEREAL BARRA COSTA FRUTOS ROJOS X 20 UDS,75263,Prisur Arbol_24_25,92,[20.0],[20.0]
5,ALOE WIN,BEBIDA ALOE WIN 500 CC ALOE VERA MANGO,12566,PRISA Arbol_24_25,BEBIDA ALOE VERA 500 ML MANGO ALOE WIN,35160,Surtiventas Arbol_24_25,92,[500.0],[500.0]
6,ALOE WIN,BEBIDA ALOE WIN 500 CC ALOE VERA MANGO,12566,PRISA Arbol_24_25,BEBIDA ALOE VERA 500 ML MANGO ALOE WIN,72475,Prisur Arbol_24_25,92,[500.0],[500.0]
7,NESCAFE,CAFE 420 GR NESCAFE TRADICION GRANULADO TARRO,89295,Prinorte Arbol_24_25,CAFE 420 GR NESCAFE TRADICIONAL GRANULADO,12596,Prisur Arbol_24_25,91,[420.0],[420.0]
8,LUCCHETTI,FIDEO 400 GR MOSTACHOLI LUCCHETTI,75555,Surtiventas Arbol_24_25,PASTA FIDEO 400 GR LUCCHETTI MOSTACHIOLI,57021,Prisur Arbol_24_25,90,[400.0],[400.0]
9,IANSA,ENDULZANTE IANSA C.K SUCRALOSA 180 ML,92474,Surtiventas Arbol_24_25,ENDULZANTE IANSA C.K ALULOSA SUCRALOSA 180 ML,91718,Prisur Arbol_24_25,90,[180.0],[180.0]


In [9]:
confident.to_excel("confident_matches.xlsx", index=False)
needs_review.to_excel("needs_review_matches.xlsx", index=False)

In [23]:
# Revisemos los productos con similitud inferior al umbral
# Ya son productos diferentes

In [24]:
excel_path = "/home/viktoria/Downloads/ARBOLES ALIMENTOS EJERCICIO KABELLI.xlsx"

_, different_products = process_excel_for_duplicates(
    excel_path,
    confidence_threshold=88,
    low_confidence_threshold=84,
)

In [25]:
different_products.shape

(38, 10)

In [27]:
different_products.head(38)

,Marca,Nombre SKU 1,SKU 1,Sheet 1,Nombre SKU 2,SKU 2,Sheet 2,Similarity,Numbers 1,Numbers 2
0,SAN JOSE,ATUN LOMITO AGUA 160 GR SAN JOSE,78799,PRISA Arbol_24_25,ATUN LOMITOS ACEITE 160 GR SAN JOSE,72495,Surtiventas Arbol_24_25,87,[160.0],[160.0]
1,SAN JOSE,ATUN LOMITOS AGUA 160 GR SAN JOSE,78799,Surtiventas Arbol_24_25,ATUN LOMITO ACEITE 160 GR SAN JOSE,74855,Prisur Arbol_24_25,87,[160.0],[160.0]
2,SAN JOSE,ATUN LOMITOS ACEITE 160 GR SAN JOSE,72495,Surtiventas Arbol_24_25,ATUN AGUA LOMITO 160 GR SAN JOSE,74854,Prisur Arbol_24_25,87,[160.0],[160.0]
3,SAN JOSE,ATUN LOMITO ACEITE 160 GR SAN JOSE,72495,PRISA Arbol_24_25,ATUN LOMITOS AGUA 160 GR SAN JOSE,78799,Surtiventas Arbol_24_25,87,[160.0],[160.0]
4,COCA-COLA,BEBIDA LATA 220CC COCA-COLA LIGHT,83484,PRISA Arbol_24_25,BEBIDA LATA 350CC COCA-COLA CERO LIGHT,82047,Prisur Arbol_24_25,87,[],[]
5,ALCAFOOD,PURE DE PAPA 250 GR ALCAFOOD,72211,Surtiventas Arbol_24_25,PURE DE PAPAS 250 GR ESTUCHE ALCAFOOD,72450,Prisur Arbol_24_25,86,[250.0],[250.0]
6,ALOE WIN,BEBIDA ALOE VERA 500 ML PI¥A ALOE WIN,35160,Surtiventas Arbol_24_25,BEBIDA ALOE VERA 500 ML MANGO ALOE WIN,72475,Prisur Arbol_24_25,86,[500.0],[500.0]
7,DULCEDIA,ALFAJOR 3 CAPAS CH/NEGRO X 24 UN X 60 GR DULCEDIA,35159,Surtiventas Arbol_24_25,ALFAJOR 3 CAPAS CH/ NEGRO 24 UN X 60 GR DULCEDIA,92321,Prisur Arbol_24_25,86,"[3.0, 24.0, 60.0]","[3.0, 24.0, 60.0]"
8,WASIL,PINAS CUBO 565 GR WASIL,52300,Surtiventas Arbol_24_25,PI¥AS EN CUBITOS 565 GR WASIL,52890,Prisur Arbol_24_25,86,[565.0],[565.0]
9,COLISEO,FIDEO 400 GR PENNE RIGATE COLISEO SPECIALE,54052,Surtiventas Arbol_24_25,PASTA FIDEO 400 GR COLISEO RIGATONI SPECIALE,72501,Prisur Arbol_24_25,86,[400.0],[400.0]
